In [1]:
from utils.openai_helpers import query_openai_model, query_openai_model_batch
from utils.preprocess_functions import convert_to_triplets
from utils.preprocess_functions import get_labels_and_descriptions_for_triplets, format_triplets_text
from utils.prompt_functions import get_prompt_HIT_test
# from utils.plotting_helpers import plot_count_distribution
import json

In [2]:
import networkx as nx
# import cugraph as cnx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

In [5]:
import utils.openai_helpers as hlp
hlp.query_openai_model('hey gpt4!, respond using JSON format')

('{\n  "response": "Hello! How can I assist you today?"\n}',
 CompletionUsage(completion_tokens=16, prompt_tokens=26, total_tokens=42))

In [3]:
with open('inputs/wikidata5m/wikidata5m_transductive_valid.txt', 'r') as file:
    wiki_valid = file.readlines()
# with open('inputs/wikidata5m/wikidata5m_transductive_train.txt', 'r') as file:
#     wiki_train = file.readlines()

In [4]:


def convert_string_to_json(string_like_json):
    # Removing the extra quotes at the beginning and the end
    clean_string = string_like_json.strip("'")
    
    # Loading the string into a JSON object
    json_object = json.loads(clean_string)
    
    return json_object

In [5]:
list_of_triplets = convert_to_triplets(wiki_valid)

In [6]:
req_list = list_of_triplets[:105]

In [7]:
triplet_list, desc_list = get_labels_and_descriptions_for_triplets(req_list)

  0%|          | 0/105 [00:00<?, ?it/s]

 42%|████▏     | 44/105 [00:39<00:35,  1.72it/s]

Error: HTTP Error 404: Not Found
Error: cannot unpack non-iterable NoneType object
Failed to get labels and descriptions for triplet: ('Q6666651', 'P17', 'Q28')


100%|██████████| 105/105 [01:30<00:00,  1.16it/s]


In [8]:
triplet_text = format_triplets_text([triplet_list[0]], [desc_list[0]])
print(triplet_text)

(Run for Cover (1955 film by Nicholas Ray), country of origin, United States of America (country primarily located in North America))


In [16]:
triplet_text

'(Run for Cover (1955 film by Nicholas Ray), country of origin, United States of America (country primarily located in North America))'

In [17]:
prompt = get_prompt_HIT_test(triplet_text)
print(prompt)


Triplet format: (subject, relation, object)
Use the given triplet and correspoding description provided below to generate two questions:
Q1) Whose answer is the object node of the triplet.
Q2) Whose answer is not the object node of the triplet, considering the triplet as a fact.

Return the two questions, their labels, along with the rationale for each generated question.
Valid labels are: "Supported" and "Refuted".

Structure your response as a JSON object with the following keys:
- "Q1": Question whose answer is the object node of the triplet.
- "Rationale_Q1": Rationale for Q1.
- "Label_Q1": Label for Q1.
- "Q2": Question whose answer is NOT the object node of the triplet.
- "Rationale_Q2": Rationale for Q2.
- "Label_Q2": Label for Q2.


The triplets and corresponding descriptions provided in brackets are as follows:
(Run for Cover (1955 film by Nicholas Ray), country of origin, United States of America (country primarily located in North America))



In [18]:
generated_text, usage = query_openai_model(prompt)

In [19]:
convert_string_to_json(generated_text)

{'Q1': 'What is the country of origin for the 1955 film Run for Cover directed by Nicholas Ray?',
 'Rationale_Q1': "The question directly asks for the country of origin of the specific film mentioned in the triplet, which is 'United States of America'. The object node of the triplet is the answer to this question, making it directly supported by the given fact.",
 'Label_Q1': 'Supported',
 'Q2': 'Was Run for Cover, a 1955 film by Nicholas Ray, primarily produced in Europe?',
 'Rationale_Q2': "The fact provided in the triplet states that 'Run for Cover (1955 film by Nicholas Ray)' has its country of origin as 'United States of America', which is primarily located in North America, not Europe. This question implies an answer that contradicts the object node of the triplet, making it refuted by the given fact.",
 'Label_Q2': 'Refuted'}

In [9]:
generated_responses = {}
prompts = []
triplets = []
descs = []

for i in range(len(triplet_list)):
    if "Error" in triplet_list[i]:
        print("Error in triplet: ", triplet_list[i])
        continue
    triplet_text = format_triplets_text([triplet_list[i]], [desc_list[i]])
    prompt = get_prompt_HIT_test(triplet_text)
    prompts.append(prompt)
    
    triplets.append(triplet_list[i])
    descs.append(desc_list[i])
    # generated_text, usage = query_openai_model(prompt)
    # generated_responses[i] = convert_string_to_json(generated_text)

Error in triplet:  ('Error', 'Error', 'Error')


In [10]:
len(descs)

104

In [11]:
np.save('outputs/test_prompts_generation.npy', prompts)

In [12]:
prompts = np.load('outputs/test_prompts_generation.npy', allow_pickle=True)

In [13]:
req_prompts = prompts[:100]
req_triplets = triplets[:100]
req_descs = descs[:100]

In [11]:
print(req_prompts[0])


Triplet format: (subject, relation, object)
Use the given triplet and correspoding description provided below to generate two questions:
Q1) Whose answer is the object node of the triplet.
Q2) Whose answer is not the object node of the triplet, considering the triplet as a fact.

Return the two questions, their labels, along with the rationale for each generated question.
Valid labels are: "Supported" and "Refuted".

Structure your response as a JSON object with the following keys:
- "Q1": Question whose answer is the object node of the triplet.
- "Rationale_Q1": Rationale for Q1.
- "Label_Q1": Label for Q1.
- "Q2": Question whose answer is NOT the object node of the triplet.
- "Rationale_Q2": Rationale for Q2.
- "Label_Q2": Label for Q2.


The triplets and corresponding descriptions provided in brackets are as follows:
(Run for Cover (1955 film by Nicholas Ray), country of origin, United States of America (country primarily located in North America))



In [14]:
results_generated = query_openai_model_batch(req_prompts, max_workers=10)
np.save('outputs/generated_responses_dict.npy', results_generated)

Processing prompts: 100%|██████████| 100/100 [05:52<00:00,  3.52s/it]


In [15]:
json_list = []
for i in range(len(results_generated)):
    json_list.append(convert_string_to_json(results_generated[i][0]))

In [16]:
# create dataframe from the generated responses
df = pd.DataFrame.from_records(json_list)

In [17]:
df

,Q1,Rationale_Q1,Label_Q1,Q2,Rationale_Q2,Label_Q2
0,What is the country of origin for the 1955 fil...,This question directly asks for the country of...,Supported,"Is 'Run for Cover', a 1955 film by Nicholas Ra...",This question is crafted to seek an answer tha...,Refuted
1,"What language did James P. Coleman, a Mississi...",The question directly asks for the language sp...,Supported,"Did James P. Coleman, the Mississippi politici...",This question implies a scenario where James P...,Refuted
2,What position did John Joseph O'Connor hold?,This question directly asks for the 'object no...,Supported,What was John Joseph O'Connor's role in the mi...,The second question seeks information that is ...,Refuted
3,What is the taxon rank of Lithops aucampiae?,This question directly asks for the taxon rank...,Supported,Is Lithops aucampiae a genus?,The question challenges the fact provided by t...,Refuted
4,What is the parent taxon of the genus Euhalidaya?,This question directly asks for the parent tax...,Supported,Which genus of insects is a member of the trib...,"This question inverts the relationship, asking...",Refuted
...,...,...,...,...,...,...
95,Who is a cast member of The New Swiss Family R...,The question directly asks for a cast member o...,Supported,Who directed The New Swiss Family Robinson (19...,This question inquires about the director of t...,Refuted
96,What sport is La Copa Junior associated with?,The question directly asks for the type of spo...,Supported,Is La Copa Junior a tournament related to amat...,This question inquires if La Copa Junior is as...,Refuted
97,What movement does 'Portrait of a Man in a Red...,The question directly asks for the classificat...,Supported,Which painting by Hans Memling is considered a...,The question implies an answer that would iden...,Refuted
98,In which province is the municipality of San E...,This question directly asks for the location (...,Supported,Is San Esteban located in the province of Cebu?,This question suggests an incorrect location (...,Refuted


In [18]:
df['triplets'] = req_triplets
df['descs'] = req_descs

In [20]:
df

,Q1,Rationale_Q1,Label_Q1,Q2,Rationale_Q2,Label_Q2,triplets,descs
0,What is the country of origin for the 1955 fil...,This question directly asks for the country of...,Supported,"Is 'Run for Cover', a 1955 film by Nicholas Ra...",This question is crafted to seek an answer tha...,Refuted,"(Run for Cover, country of origin, United Stat...","(1955 film by Nicholas Ray, country of origin ..."
1,"What language did James P. Coleman, a Mississi...",The question directly asks for the language sp...,Supported,"Did James P. Coleman, the Mississippi politici...",This question implies a scenario where James P...,Refuted,"(James P. Coleman, languages spoken, written o...","(Mississippi politician (1914-1991), language(..."
2,What position did John Joseph O'Connor hold?,This question directly asks for the 'object no...,Supported,What was John Joseph O'Connor's role in the mi...,The second question seeks information that is ...,Refuted,"(John Joseph O'Connor, position held, archbishop)","(American Cardinal, Archbishop of New York, su..."
3,What is the taxon rank of Lithops aucampiae?,This question directly asks for the taxon rank...,Supported,Is Lithops aucampiae a genus?,The question challenges the fact provided by t...,Refuted,"(Lithops aucampiae, taxon rank, species)","(species of plant, level in a taxonomic hierar..."
4,What is the parent taxon of the genus Euhalidaya?,This question directly asks for the parent tax...,Supported,Which genus of insects is a member of the trib...,"This question inverts the relationship, asking...",Refuted,"(Euhalidaya, parent taxon, Blondeliini)","(genus of insects, closest parent taxon of the..."
...,...,...,...,...,...,...,...,...
95,Who is a cast member of The New Swiss Family R...,The question directly asks for a cast member o...,Supported,Who directed The New Swiss Family Robinson (19...,This question inquires about the director of t...,Refuted,"(The New Swiss Family Robinson, cast member, J...","(1998 film by Stewart Raffill, actor in the su..."
96,What sport is La Copa Junior associated with?,The question directly asks for the type of spo...,Supported,Is La Copa Junior a tournament related to amat...,This question inquires if La Copa Junior is as...,Refuted,"(La Copa Junior, sport, professional wrestling)","(Mexican professional wrestling tournament, sp..."
97,What movement does 'Portrait of a Man in a Red...,The question directly asks for the classificat...,Supported,Which painting by Hans Memling is considered a...,The question implies an answer that would iden...,Refuted,"(Portrait of a Man in a Red Hat, movement, Ear...","(painting by Hans Memling, literary, artistic,..."
98,In which province is the municipality of San E...,This question directly asks for the location (...,Supported,Is San Esteban located in the province of Cebu?,This question suggests an incorrect location (...,Refuted,"(San Esteban, located in the administrative te...",(municipality of the Philippines in the provin...


In [21]:
df.to_csv('outputs/QA_1hop.csv')

In [28]:
def generate_sagemaker_groundtruth_input(csv_file_path, output_json_path):
    # Load the CSV file
    data = pd.read_csv(csv_file_path)

    # Create a list to store JSON records
    records = []

    # Loop through each row in the dataframe and create a JSON record
    for index, row in data.iterrows():
        # Parse the triplet to extract the object node (answer)
        triplet = eval(row["triplets"])
        answer = triplet[2]
        
        record = {
            "source": {
                "question": row["Q1"],
                "triplet": row["triplets"],
                "answer": answer,
                "assigned_label": row["Label_Q1"]
            }
        }
        records.append(record)

    # Write the records to a JSON file
    with open(output_json_path, 'w') as json_file:
        for record in records:
            json_file.write(json.dumps(record) + '\n')


In [29]:
generate_sagemaker_groundtruth_input('outputs/QA_1hop.csv', 'outputs/QA_1hop_sage.json')

In [30]:
#open the json file and check the format
with open('outputs/QA_1hop_sage.json', 'r') as file:
    data = file.readlines()
    

In [56]:
def generate_sagemaker_formatted_csv(csv_file_path, output_csv_path):
    # Load the CSV file
    data = pd.read_csv(csv_file_path)

    # Create a list to store CSV records
    records = []

    # Loop through each row in the dataframe and create a CSV record
    for index, row in data.iterrows():
        # Parse the triplet to extract the object node (answer)
        triplet = eval(row["triplets"])
        answer = triplet[2]
        
        record = {
            "question": f"*** QUESTION ***: {row['Q1']}",
            "triplet": f"*** TRIPLET ***: {row['triplets']}",
            "answer": f"*** ANSWER ***: {answer}",
            "assigned_label": f"*** PREDICTION ***: {row['Label_Q1']}"
        }
        records.append(record)

    # Convert the records list to a DataFrame
    output_df = pd.DataFrame(records)

    # Write the DataFrame to a CSV file
    output_df.to_csv(output_csv_path, index=False)


In [52]:
def generate_sagemaker_groundtruth_input_csv(csv_file_path, output_csv_path):
    # Load the CSV file
    data = pd.read_csv(csv_file_path)

    # Create a list to store CSV records
    records = []

    # Loop through each row in the dataframe and create a CSV record
    for index, row in data.iterrows():
        # Parse the triplet to extract the object node (answer)
        triplet = eval(row["triplets"])
        answer = triplet[2]
        
        # Create the record in the format expected by SageMaker Ground Truth
        record = {
            "taskInput": {
                "question": f"Question: {row['Q1']}",
                "triplet": f"Triplet: {row['triplets']}",
                "answer": f"Answer: {answer}",
                "assigned_label": f"Assigned Label: {row['Label_Q1']}"
            }
        }
        records.append(record)

    # Convert the records list to a DataFrame
    output_df = pd.DataFrame(records)

    # Write the DataFrame to a CSV file
    output_df.to_csv(output_csv_path, index=False, quoting=csv.QUOTE_ALL)

In [57]:

generate_sagemaker_formatted_csv('outputs/QA_1hop.csv', 'outputs/QA_1hop_sagemaker_formatted_v1.csv')

In [53]:
generate_sagemaker_groundtruth_input_csv('outputs/QA_1hop.csv', 'outputs/QA_1hop_sage_formatted_v1.csv')

In [1]:
wikidata_properties = {
    "Geospatial Information": {
        "Moving Up the Hierarchy": {
            "Located in the administrative territorial entity": "P131",
            "Part of": "P361",
            "Located on terrain feature": "P706"
        },
        "Moving Down the Hierarchy": {
            "Contains administrative territorial entity": "P150",
            "Location": "P276",
            "Located on street": "P669"
        },
        "Lateral Connections": {
            "Location": "P276",
            "Located on terrain feature": "P706"
        },
        "General Context": {
            "Country": "P17",
            "Instance of": "P31"
        }
    },
    "Occupation and Professional Information": {
        "Moving Up the Hierarchy": {
            "Field of work": "P101",
            "Part of": "P361",
            "Academic discipline": "P136"
        },
        "Moving Down the Hierarchy": {
            "Has part": "P527",
            "Subclass of": "P279"
        },
        "Lateral Connections": {
            "Employer": "P108",
            "Member of": "P463",
            "Affiliation": "P1416"
        },
        "General Context": {
            "Occupation": "P106",
            "Position held": "P39"
        }
    },
    "Temporal Information": {
        "Moving Up the Hierarchy": {
            "Follows": "P155",
            "Part of": "P361"
        },
        "Moving Down the Hierarchy": {
            "Has part": "P527",
            "Subclass of": "P279"
        },
        "Lateral Connections": {
            "Concurrent with": "P1072",
            "Followed by": "P156"
        },
        "General Context": {
            "Point in time": "P585",
            "Duration": "P2047"
        }
    },
    "Biological and Taxonomic Information": {
        "Moving Up the Hierarchy": {
            "Parent taxon": "P171",
            "Part of": "P361"
        },
        "Moving Down the Hierarchy": {
            "Taxon rank": "P105",
            "Has part": "P527"
        },
        "Lateral Connections": {
            "Related to": "P2789",
            "Similar to": "P1889"
        },
        "General Context": {
            "Instance of": "P31",
            "Species": "P105"
        }
    },
    "Cultural and Creative Work": {
        "Moving Up the Hierarchy": {
            "Part of the series": "P179",
            "Genre": "P136"
        },
        "Moving Down the Hierarchy": {
            "Has part": "P527",
            "Subgenre": "P741"
        },
        "Lateral Connections": {
            "Inspired by": "P941",
            "Derivative work": "P4969"
        },
        "General Context": {
            "Instance of": "P31",
            "Creative work type": "P1448"
        }
    },
    "Historical and Event Information": {
        "Moving Up the Hierarchy": {
            "Part of": "P361",
            "Significant event": "P793"
        },
        "Moving Down the Hierarchy": {
            "Has part": "P527",
            "Instance of": "P31"
        },
        "Lateral Connections": {
            "Followed by": "P156",
            "Follows": "P155"
        },
        "General Context": {
            "Point in time": "P585",
            "Location": "P276"
        }
    },
    "Organizational Information": {
        "Moving Up the Hierarchy": {
            "Parent organization": "P749",
            "Part of": "P361"
        },
        "Moving Down the Hierarchy": {
            "Subsidiary": "P355",
            "Branch": "P1833"
        },
        "Lateral Connections": {
            "Affiliate": "P1416",
            "Member of": "P463"
        },
        "General Context": {
            "Industry": "P452",
            "Instance of": "P31"
        }
    },
    "Literary and Bibliographic Information": {
        "Moving Up the Hierarchy": {
            "Part of the series": "P179",
            "Genre": "P136"
        },
        "Moving Down the Hierarchy": {
            "Has part": "P527",
            "Subgenre": "P741"
        },
        "Lateral Connections": {
            "Inspired by": "P941",
            "Derivative work": "P4969"
        },
        "General Context": {
            "Instance of": "P31",
            "Author": "P50"
        }
    },
    "Scientific and Technical Information": {
        "Moving Up the Hierarchy": {
            "Field of work": "P101",
            "Part of": "P361"
        },
        "Moving Down the Hierarchy": {
            "Has part": "P527",
            "Subclass of": "P279"
        },
        "Lateral Connections": {
            "Related to": "P2789",
            "Similar to": "P1889"
        },
        "General Context": {
            "Instance of": "P31",
            "Research topic": "P2579"
        }
    },
    "Social and Demographic Information": {
        "Moving Up the Hierarchy": {
            "Part of": "P361",
            "Member of": "P463"
        },
        "Moving Down the Hierarchy": {
            "Subclass of": "P279",
            "Has part": "P527"
        },
        "Lateral Connections": {
            "Related to": "P2789",
            "Similar to": "P1889"
        },
        "General Context": {
            "Instance of": "P31",
            "Demographic group": "P1448"
        }
    }
}



In [2]:

for key, value in wikidata_properties["Occupation and Professional Information"]["Moving Up the Hierarchy"].items():
    print(f"{key}: {value}")


Field of work: P101
Part of: P361
Academic discipline: P136
